<a href="https://colab.research.google.com/github/deepshah1300/ML/blob/master/Exp12_J077.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget -O AirQualityUCI.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip
df = pd.read_excel('/content/AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']])
df.replace(to_replace=-200.0, value=0.0, inplace=True)
wv = df['NMHC(GT)']
bad_wv = wv == -200.0
wv[bad_wv] = 0.0

--2020-10-31 08:24:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1543989 (1.5M) [application/x-httpd-php]
Saving to: ‘AirQualityUCI.zip’

AirQualityUCI.zip   100%[===================>]   1.47M  1.88MB/s    in 0.8s    

2020-10-31 08:24:22 (1.88 MB/s) - ‘AirQualityUCI.zip’ saved [1543989/1543989]

Archive:  AirQualityUCI.zip
  inflating: AirQualityUCI.csv       
  inflating: AirQualityUCI.xlsx      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [3]:
target = 'NO2(GT)'
target

'NO2(GT)'

In [4]:
df.corr()[['NO2(GT)']]

,NO2(GT)
CO(GT),0.723154
PT08.S1(CO),0.284508
NMHC(GT),0.099541
C6H6(GT),0.402581
PT08.S2(NMHC),0.334108
NOx(GT),0.795888
PT08.S3(NOx),-0.440202
NO2(GT),1.000000
PT08.S4(NO2),0.010185
PT08.S5(O3),0.439057


In [5]:
df.head()
df = df[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)','PT08.S2(NMHC)', 'NOx(GT)','NO2(GT)','PT08.S5(O3)']]

In [6]:
df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),NO2(GT),PT08.S5(O3)
0,2.6,1360.00,150,1045.50,166.0,113.0,1267.50
1,2.0,1292.25,112,954.75,103.0,92.0,972.25
2,2.2,1402.00,88,939.25,131.0,114.0,1074.00
3,2.2,1375.50,80,948.25,172.0,122.0,1203.25
4,1.6,1272.25,51,835.50,131.0,116.0,1110.00


In [7]:
X = df.drop(target,axis=1)
y = df[[target]]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y)
value = y_test

In [9]:
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(7017, 6)
(2340, 6)
(7017, 1)
(2340, 1)


In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.fit_transform(x_train)
scaler.fit(x_test)
x_test = scaler.fit_transform(x_test)
scaler.fit(y_train)
y_train = scaler.fit_transform(y_train)
scaler.fit(y_test)
y_test = scaler.fit_transform(y_test)

In [12]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
print (x_train.shape)
print (x_test.shape)

(7017, 6, 1)
(2340, 6, 1)


In [13]:
print (y_train.shape)
print (y_test.shape)

(7017, 1)
(2340, 1)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
def RNN_MODEL():
    # have to convert to 3D for feeding the data
    regressor = Sequential()
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['accuracy'])
    return regressor

In [15]:
regressor = RNN_MODEL()

In [16]:
regressor.fit(x_train,y_train,epochs = 100)

Epoch 1/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0261 - accuracy: 0.1804
Epoch 2/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0214 - accuracy: 0.1803
Epoch 3/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0189 - accuracy: 0.1801
Epoch 4/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0174 - accuracy: 0.1800
Epoch 5/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0161 - accuracy: 0.1804
Epoch 6/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0159 - accuracy: 0.1800
Epoch 7/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0153 - accuracy: 0.1798
Epoch 8/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0153 - accuracy: 0.1806
Epoch 9/100
220/220 [==============================] - 2s 11ms/step - loss: 0.0146 - accuracy: 0.1804
Epoch 10/100
220/220 [==============================] - 2s 11ms/step - loss: 0.014

In [17]:
regressor.evaluate(x_test,y_test)

74/74 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 0.1611


[0.0059156836941838264, 0.16111111640930176]

In [18]:
y_pred = regressor.predict(x_test)

In [19]:
output = scaler.inverse_transform(y_pred)
real_output = []
for item in output:
    real_output.append((item[0]))

In [20]:
real_output

[156.83566,
 173.72011,
 154.01202,
 185.59485,
 130.17088,
 121.233696,
 131.88013,
 178.67798,
 95.36658,
 107.168594,
 49.292786,
 -4.0937304,
 102.555756,
 157.24983,
 197.80527,
 143.52376,
 147.10126,
 102.163666,
 103.43081,
 97.79061,
 60.137695,
 81.38517,
 168.12341,
 157.6991,
 -3.7857072,
 197.13683,
 185.0466,
 139.39104,
 96.927666,
 57.735947,
 -4.0985675,
 166.08772,
 106.3722,
 180.0913,
 142.62007,
 192.30254,
 53.391148,
 110.841774,
 42.98474,
 69.102066,
 148.10205,
 -2.3029928,
 -4.1600633,
 54.76829,
 108.86184,
 58.65509,
 74.02277,
 190.00479,
 64.324776,
 125.877815,
 232.52827,
 59.342228,
 190.93541,
 65.3947,
 145.61113,
 -1.1586729,
 145.32864,
 -4.3402877,
 218.50854,
 138.44748,
 61.8519,
 128.39923,
 104.99248,
 195.48279,
 168.05031,
 77.77315,
 135.03046,
 174.47565,
 148.04819,
 128.71107,
 95.42121,
 77.61435,
 173.327,
 84.425644,
 -1.4030694,
 137.26086,
 70.27848,
 -4.11865,
 63.66913,
 184.3372,
 74.624985,
 97.69252,
 128.47946,
 50.582233,
 22

In [21]:
value['predicted'] = np.array(real_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
import matplotlib.pyplot as plt

In [23]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (100,100)

In [24]:
value.plot()